In [25]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
# from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
# from textblob import TextBlob
# from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os
import warnings
warnings.filterwarnings('ignore')

In [11]:
text = pd.read_csv('/home/rupeek/Desktop/ML &AI/NLP_archive/IMDB Dataset.csv')
text.head()
txt = text[0:10000]

In [12]:
txt.describe()


,review,sentiment
count,10000,10000
unique,9983,2
top,I want very much to believe that the above quo...,positive
freq,2,5028


In [13]:
#sentiment count
txt['sentiment'].value_counts()

positive    5028
negative    4972
Name: sentiment, dtype: int64

## Spliting the training dataset



In [18]:
#split the dataset  
#train dataset
train_reviews=txt.review[:6000]
train_sentiments=txt.sentiment[:6000]
#test dataset
test_reviews=txt.review[6000:]
test_sentiments=txt.sentiment[6000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(6000,) (6000,)
(4000,) (4000,)


## Text normalization



In [26]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

Removing html strips and noise text



In [28]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
txt['review']=txt['review'].apply(denoise_text)

Removing special characters

In [30]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
txt['review']=txt['review'].apply(remove_special_characters)

## Text stemming



In [31]:
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
txt['review']=txt['review'].apply(simple_stemmer)

Removing stopwords



In [32]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
txt['review']=txt['review'].apply(remove_stopwords)

{'in', 'very', 'over', "hadn't", 'but', 'there', 'theirs', 'you', 'have', 'i', 'd', 're', 'other', 'above', 'of', 't', 'because', 'yours', "doesn't", 'wouldn', 'each', 'under', 'how', 'wasn', 'aren', 'below', "you're", 'didn', 'hasn', 'weren', 'y', 'here', 'or', "hasn't", 'they', 'our', 'it', 'll', 'on', 'should', 'his', 've', "you've", 'some', 'during', 'off', 'once', "mustn't", 'be', 'my', 'he', 'a', 'm', "she's", 'against', 'was', 'doing', 'we', 'myself', 'into', 'itself', "wasn't", 'for', 'yourselves', 'your', 'until', 'after', 'which', 'about', 'down', 'him', 'needn', 'won', 'whom', 'ma', "should've", 'up', 'too', 'themselves', 'can', 'isn', "shouldn't", 'what', 'out', "didn't", 'that', 'its', 'if', 'same', 'herself', "shan't", 'having', 'has', 'himself', 'through', 'hers', 'why', "mightn't", 'before', 'where', "you'd", 'not', 'own', "couldn't", 'o', 'am', 'and', 'both', 'me', 'no', 'did', 'between', 'ours', 'again', 'she', "wouldn't", "it's", 'only', 'more', 'were', "won't", 'wil

Normalized train reviews



In [35]:
#normalized train reviews
norm_train_reviews=txt.review[:6000]
norm_train_reviews[0]

'one review ha mention watch 1 Oz episod youll hook right thi exactli happen meth first thing struck Oz wa brutal unflinch scene violenc set right word GO trust thi show faint heart timid thi show pull punch regard drug sex violenc hardcor classic use wordit call OZ nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda Em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awayi would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti wa surreal couldnt say wa readi watch develop tast Oz got accustom high level graphic violenc violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison exp

Normalized test reviews



In [37]:
norm_test_reviews=txt.review[4000:]
norm_test_reviews[4005]

'movi thought wa excel wa suppos romanc littl suspens betweenrob stewart wonder actor dont know whi peopl keep give bad rap mel harri great actress think look old rob onli five yearsrob lead role hi TV seri well one scifi channel Im sure rememb topic heat aka sweat bullet painkil janeh also star number TV movi make TV mini seriesthey need give lead role best'

Bags of words model

It is used to convert text documents to numerical vectors or bag of words.



In [38]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(norm_train_reviews)
#transformed test reviews
cv_test_reviews=cv.transform(norm_test_reviews)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)
#vocab=cv.get_feature_names()-toget feature names

BOW_cv_train: (6000, 1167543)
BOW_cv_test: (6000, 1167543)


Term Frequency-Inverse Document Frequency model (TFIDF)

It is used to convert text documents to matrix of tfidf features.



In [40]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(norm_train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(norm_test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (6000, 1167543)
Tfidf_test: (6000, 1167543)


Labeling the sentiment text



In [41]:
#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(txt['sentiment'])
print(sentiment_data.shape)

(10000, 1)


In [42]:
#Spliting the sentiment data
train_sentiments=sentiment_data[:6000]
test_sentiments=sentiment_data[4000:]
print(train_sentiments)
print(test_sentiments)

[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [1]]
[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [1]]


In [43]:
#training the linear svm
svm=SGDClassifier(loss='hinge',max_iter=500,random_state=42)
#fitting the svm for bag of words
svm_bow=svm.fit(cv_train_reviews,train_sentiments)
print(svm_bow)
#fitting the svm for tfidf features
svm_tfidf=svm.fit(tv_train_reviews,train_sentiments)
print(svm_tfidf)

SGDClassifier(max_iter=500, random_state=42)
SGDClassifier(max_iter=500, random_state=42)


Model performance on test data




In [45]:
#Predicting the model for bag of words
svm_bow_predict=svm.predict(cv_test_reviews)
print(svm_bow_predict)
#Predicting the model for tfidf features
svm_tfidf_predict=svm.predict(tv_test_reviews)
print(svm_tfidf_predict)

[1 1 1 ... 0 0 1]
[1 1 1 ... 0 0 1]
